In [1]:
!pip install fastai --upgrade
from fastai.vision.all import *

     |████████████████████████████████| 358kB 2.7MB/s 
  Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [ ]:
from zipfile import ZipFile

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00486/rice_leaf_diseases.zip
!mkdir path

with ZipFile('rice_leaf_diseases.zip','r') as zip:
  zip.extractall(path='path/')

In [ ]:
diseases = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(224))

diseases = diseases.new(item_tfms=Resize(224), batch_tfms=aug_transforms(mult=2))
dls = diseases.dataloaders('path')

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=error_rate)

In [ ]:
learn.fine_tune(10,3e-4)

In [ ]:
from ipywidgets import widgets
from ipywidgets import VBox

btn_upload = widgets.FileUpload()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
btn_run = widgets.Button(description='Classify')

VBox([widgets.Label('Select rice disease image'), 
      btn_upload, btn_run, out_pl, lbl_pred])

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)    